In [317]:
import pandas as pd
import quandl
import math, datetime
import numpy as np
from sklearn import preprocessing, cross_validation, svm
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from matplotlib import style

style.use('ggplot')

In [318]:
df = quandl.get("SSE/GGQ1", authtoken="Cas8VhuiMzJszKz9XDEb")
df.head()

ProxyError: HTTPSConnectionPool(host='www.quandl.com', port=443): Max retries exceeded with url: /api/v3/datasets/SSE/GGQ1/data?order=asc (Caused by ProxyError('Cannot connect to proxy.', RemoteDisconnected('Remote end closed connection without response',)))

In [ ]:
df = pd.read_csv('SSE-GGQ1.csv',index_col='Date', parse_dates = True)
df.head()

In [ ]:
df = df[['High','Low','Last','Previous Day Price','Volume']]

In [ ]:
df.tail()

In [ ]:
df['HL_PCT'] = (df['High'] - df['Low'])/ df['Low'] *  100.0
df['%CHG_DAILY'] = (df['Last'] - df['Previous Day Price'])/ df['Previous Day Price'] *  100.0

In [ ]:
forecast_col = 'Last'
df.fillna(-99999, inplace = True)

forecast_out = int(math.ceil(0.1*len(df)))
df['Label']= df[forecast_col].shift(-forecast_out)
df.dropna(inplace = True)
print(forecast_out)

In [ ]:
X = np.array(df.drop(['Label'],1))
X = preprocessing.scale(X)
X_lately = X[-forecast_out:]
X = X[:-forecast_out]

df.dropna(inplace = True)
y = np.array(df['Label'])
y = np.array(df['Label'])
y = y[:-forecast_out]

X_train, X_test, y_train, y_test = cross_validation.train_test_split(X,y, test_size= 0.2)

clf = LinearRegression()
accuracy = clf.score(X_test, y_test)

#print(accuracy)
forecast_set = clf.predict(X_lately)
print(forecast_set, accuracy, forecast_out)


In [ ]:
df['Forecast'] = np.nan

last_date = df.iloc[-1].name
last_unix = last_date.timestamp()
one_day = 86400
next_unix = last_unix + one_day

for i in forecast_set:
    next_date = datetime.datetime.fromtimestamp(next_unix)
    next_unix += one_day
    df.loc[next_date] = [np.nan for __ in range(len(df.columns)-1)] + [i]
    
print(df.head())    

In [ ]:
df['Last'].plot()
df['Forecast'].plot()
plt.legend(loc=4)
plt.xlabel('Date')
plt.ylabel('Price')
plt.show()